## Homework: Vector Search
In this homework, we'll experiemnt with vector with and without Elasticsearch

## Q1. Getting the embeddings model

First, we will get the embeddings model ```multi-qa-distilbert-cos-v1```from the [Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)


In [1]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

```Create the embedding for this user question:```

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [29]:
q = model.encode(user_question)

len(q)

768

In [30]:
q

array([ 7.82226548e-02, -4.01311405e-02,  3.86135913e-02, -1.78966438e-04,
        8.92347097e-02, -5.04591092e-02, -1.05026569e-02,  3.71055678e-02,
       -4.18713912e-02,  3.48084792e-02, -1.20701883e-02, -2.36942340e-02,
        3.87900174e-02,  1.60988607e-02,  3.50747295e-02,  3.04746162e-03,
        5.79672381e-02, -4.10627462e-02, -3.41552682e-02, -2.56396383e-02,
       -3.55263911e-02,  1.42908087e-02, -1.62799917e-02,  3.21446545e-02,
       -4.66897376e-02,  7.89186060e-02,  4.90160920e-02,  1.56761166e-02,
       -1.69110075e-02,  2.26482227e-02,  5.60206100e-02, -3.98361087e-02,
        6.77409917e-02, -1.20209912e-02,  1.12621894e-03, -1.94394365e-02,
       -2.65951678e-02,  1.06177367e-02,  1.69687122e-02,  1.13487840e-02,
       -2.97063086e-02,  5.25258258e-02, -1.41453547e-02,  4.61699851e-02,
        1.17066065e-02, -2.38053519e-02, -6.32558241e-02, -1.92042235e-02,
       -7.10592186e-03,  3.24167833e-02,  2.49618199e-02, -5.27503015e-03,
        2.01149024e-02, -

In [31]:
result[0]

0.078222655

### Question:
What's the first value of the resulting vector?

- -0.24
- -0.04
- 0.07
- 0.27

### Answer:
- 0.07

## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [9]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [15]:
documents[0]['course'] 

'data-engineering-zoomcamp'

In [16]:
len(documents)

948

In [20]:
filter_doc = [i for i in documents if i['course'] == 'machine-learning-zoomcamp']

filter_doc

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?'

In [21]:
len(filter_doc)

375

## Q2. Creating the embeddings


Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix ```X```:

- Create a list ```embeddings```
- Iterate over each document
- ```qa_text = f'{question} {text}'```
- compute the embedding for```qa_text```, append to ```embeddings```
- At the end, let ```X = np.array(embeddings)``` ```(import numpy as np)```

What's the shape of X? ```(X.shape```. Include the parantheses.

In [23]:
from tqdm.auto import tqdm

embeddings = []

for i in tqdm(filter_doc):
    question = i['question']
    text = i['text']
    qa_text = f'{question} {text}'
    emb_qa_text = model.encode(qa_text)
    embeddings.append(emb_qa_text)

100%|██████████████████████████████████████████████████████████████████████| 375/375 [01:05<00:00,  5.75it/s]


```Put all into one single matrix```

In [26]:
import numpy as np

X = np.array(embeddings)

In [27]:
X

array([[ 0.08805905,  0.01559355,  0.07925582, ...,  0.00745118,
         0.00241914,  0.01147128],
       [ 0.07066919, -0.01930568,  0.07738373, ...,  0.04495561,
         0.01078552, -0.02316595],
       [ 0.10229158, -0.01663243,  0.03414485, ...,  0.03215314,
        -0.04317448, -0.04574372],
       ...,
       [-0.00768192,  0.0107567 ,  0.01190489, ...,  0.06066168,
        -0.03393206,  0.01605331],
       [ 0.13408281, -0.03755205,  0.01976525, ...,  0.02163627,
        -0.01921137,  0.03690192],
       [ 0.0438602 , -0.0130077 ,  0.06373122, ...,  0.03338452,
         0.0225854 , -0.06305878]], dtype=float32)

In [28]:
X.shape

(375, 768)

### Answer:
- (375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it ```v```) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector ```v```:

In [38]:
q.dot(q)

0.9999999

In [33]:
scores = X.dot(q)

### Question: 

What's the highest score in the results?

- 65.0
- 6.5
- 0.65
- 0.065

In [37]:
scores.max()

0.6506573

### Answer:
- 0.65

## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [42]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filter_doc, embeddings=X)
search_engine.search(q, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

If you don't understand how the search function work:

- Ask ChatGTP or any other LLM of your choice to explain the code
- Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the ```hit rate``` metric for evaluation.

First, load the ground truth dataset:

In [43]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the ```hit rate``` of VectorSearchEngine with num_results=5.

What did you get?

- 0.93
- 0.73
- 0.53
- 0.33

In [54]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [55]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [59]:
def search_engine(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    search_engine = VectorSearchEngine(documents=documents, embeddings=v_q)

    return res

In [ ]:
search_engine = VectorSearchEngine(documents=documents, embeddings=embeddings)


In [60]:
evaluate(ground_truth, search_engine)


  0%|                                                                               | 0/1830 [00:00<?, ?it/s]


AttributeError: 'function' object has no attribute 'search'